In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import numpy as np
import pandas as pd
import math


import seaborn as sns
import acquire as a
import prepare as p
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = a.get_curriculum_logs_data()

In [3]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 847330 entries, 0 to 847329
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        847330 non-null  object 
 1   time        847330 non-null  object 
 2   path        847329 non-null  object 
 3   user_id     847330 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          847330 non-null  object 
 6   id          847330 non-null  int64  
 7   name        847330 non-null  object 
 8   slack       847330 non-null  object 
 9   start_date  847330 non-null  object 
 10  end_date    847330 non-null  object 
 11  created_at  847330 non-null  object 
 12  updated_at  847330 non-null  object 
 13  deleted_at  0 non-null       float64
 14  program_id  847330 non-null  int64  
dtypes: float64(2), int64(3), object(10)
memory usage: 103.4+ MB


In [5]:
df.shape

(847330, 15)

In [6]:
df.duplicated().sum()

3518

In [7]:
df.date = pd.to_datetime(df.date)
df = df.set_index(df.date)

In [9]:
df = df.drop(columns=['date', 'id'])

In [11]:
df.start_date = pd.to_datetime(df.start_date)
df.end_date = pd.to_datetime(df.end_date)

In [12]:
df = df.rename(columns={'path':'endpoint',
                       'name':'cohort'})

In [13]:
df = df.drop(columns=['id'])

In [14]:
df.head()

,time,endpoint,user_id,cohort_id,ip,cohort,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
date,,,,,,,,,,,,,
2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2


In [7]:
# df = p.prep_logs(df)

There are 3518 duplicate rows in your dataset - these will be dropped.
----------------


In [15]:
df = df.drop_duplicates()

In [16]:
df.shape

(843553, 13)

### question 1

In [17]:
df.program_id.value_counts()

2    710296
3    102820
1     30432
4         5
Name: program_id, dtype: int64

In [19]:
df.groupby('program_id').endpoint.value_counts()

program_id  endpoint                                   
1           /                                              1681
            index.html                                     1011
            javascript-i                                    735
            html-css                                        540
            spring                                          496
                                                           ... 
3           where                                             1
4           content/html-css                                  2
            /                                                 1
            content/html-css/gitbook/images/favicon.ico       1
            content/html-css/introduction.html                1
Name: endpoint, Length: 3309, dtype: int64

In [14]:
df.endpoint[(df.program_id == 3)].value_counts()

/                                           8240
search/search_index.json                    2187
classification/overview                     1783
1-fundamentals/1.1-intro-to-data-science    1632
1-fundamentals/modern-data-scientist.jpg    1613
                                            ... 
.gitignore                                     1
2-storytelling/1-overview/www.qlik.com         1
capstones                                      1
appendix/open_data/www.census.gov              1
12-distributed-ml/4-acquire                    1
Name: endpoint, Length: 682, dtype: int64

In [15]:
df.endpoint[(df.program_id == 2)].value_counts()

/                                                           35765
javascript-i                                                17403
toc                                                         17338
search/search_index.json                                    15171
java-iii                                                    12628
                                                            ...  
content/mysql/sample-database/gitbook/images/favicon.ico        1
4-python/error-handling                                         1
capstone/55                                                     1
bayes-capstones                                                 1
9-anomaly-detection/time-series-anomaly-detection-part-1        1
Name: endpoint, Length: 1913, dtype: int64

In [21]:
df.endpoint[(df.program_id == 4)].value_counts()

content/html-css                               2
content/html-css/introduction.html             1
/                                              1
content/html-css/gitbook/images/favicon.ico    1
Name: endpoint, dtype: int64

In [22]:
df.endpoint[(df.program_id == 1)].value_counts()

/                                                              1681
index.html                                                     1011
javascript-i                                                    735
html-css                                                        540
spring                                                          496
                                                               ... 
1-fundamentals/2.4-more-excel-features                            1
content/jquery/essential-methods/traversing.html                  1
html/forms                                                        1
content/examples/git/gitbook/images/favicon.ico                   1
content/appendix/javascript/javascript/functions/scope.html       1
Name: endpoint, Length: 710, dtype: int64

### Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [18]:
df.head()

,time,endpoint,user_id,cohort_id,ip,cohort,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
date,,,,,,,,,,,,,
2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2


In [21]:
df.groupby('cohort_id').endpoint.agg(['count','nunique'])


,count,nunique
cohort_id,,
1.0,8830,342
2.0,93,44
4.0,4,4
5.0,1,1
6.0,72,45
7.0,597,143
8.0,1712,278
9.0,5,4
11.0,253,95


In [38]:
df.groupby('cohort_id').endpoint.value_counts()

cohort_id  endpoint                                   
1.0        /                                              626
           javascript-i                                   293
           html-css                                       213
           javascript-ii                                  201
           spring                                         187
                                                         ... 
139.0      java-iii/servlets                                1
           javascript-i/bom-and-dom/dom                     1
           javascript-i/objects                             1
           javascript-i/objects/math                        1
           jquery/essential-methods/attributes-and-css      1
Name: endpoint, Length: 13565, dtype: int64

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 843553 entries, 2018-01-26 to 2021-04-21
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        843553 non-null  object        
 1   endpoint    843552 non-null  object        
 2   user_id     843553 non-null  int64         
 3   cohort_id   843553 non-null  float64       
 4   ip          843553 non-null  object        
 5   cohort      843553 non-null  object        
 6   slack       843553 non-null  object        
 7   start_date  843553 non-null  datetime64[ns]
 8   end_date    843553 non-null  datetime64[ns]
 9   created_at  843553 non-null  object        
 10  updated_at  843553 non-null  object        
 11  deleted_at  0 non-null       float64       
 12  program_id  843553 non-null  int64         
dtypes: datetime64[ns](2), float64(2), int64(2), object(7)
memory usage: 90.1+ MB


In [46]:
new = df[['cohort_id', 'endpoint']]
new.head()

,cohort_id,endpoint
date,,
2018-01-26,8.0,/
2018-01-26,8.0,java-ii
2018-01-26,8.0,java-ii/object-oriented-programming
2018-01-26,8.0,slides/object_oriented_programming
2018-01-26,22.0,javascript-i/conditionals


In [52]:
# set to display all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
(df.cohort_id == 1)

In [42]:
df.endpoint

date
2018-01-26                                                   /
2018-01-26                                             java-ii
2018-01-26                 java-ii/object-oriented-programming
2018-01-26                  slides/object_oriented_programming
2018-01-26                           javascript-i/conditionals
                                    ...                       
2021-04-21                                jquery/personal-site
2021-04-21                                   jquery/mapbox-api
2021-04-21                             jquery/ajax/weather-map
2021-04-21    anomaly-detection/discrete-probabilistic-methods
2021-04-21                                   jquery/mapbox-api
Name: endpoint, Length: 843553, dtype: object

3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [61]:
df['is_active'] = (df.index >= df.start_date) & (df.index <= df.end_date)

In [64]:
df1 = df.sample(50)

In [68]:
active_stu = df1[df1.is_active == True]
active_stu.head()

,time,endpoint,user_id,cohort_id,ip,cohort,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,is_active
date,,,,,,,,,,,,,,
2021-02-03,07:54:47,javascript-i,869,135.0,136.50.73.42,Marco,#marco,2021-01-25,2021-07-19,2021-01-20 21:31:11,2021-01-20 21:31:11,NaN,2,True
2019-05-24,12:10:29,appendix/extra-challenges,375,31.0,97.105.19.58,Andromeda,#andromeda,2019-03-18,2019-07-30,2019-03-18 20:35:06,2019-03-18 20:35:06,NaN,2,True
2018-03-14,01:20:12,java-i,5,22.0,24.155.251.203,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,True
2020-03-05,16:24:41,4-python/8.4.3-dataframes,581,55.0,97.105.19.58,Curie,#curie,2020-02-03,2020-07-07,2020-02-03 19:31:51,2020-02-03 19:31:51,NaN,3,True
2021-02-14,08:35:29,spring/fundamentals/security/authentication,764,62.0,72.190.174.52,Jupiter,#jupiter,2020-09-21,2021-03-30,2020-09-21 18:06:27,2020-09-21 18:06:27,NaN,2,True


In [71]:
active_stu.groupby('cohort_id').sum()

,user_id,deleted_at,program_id,is_active
cohort_id,,,,
22.0,18,0.0,4,2
23.0,131,0.0,2,1
24.0,383,0.0,4,2
25.0,231,0.0,2,1
26.0,269,0.0,2,1
27.0,301,0.0,2,1
29.0,672,0.0,4,2
31.0,1195,0.0,6,3
32.0,838,0.0,4,2


In [83]:
df.groupby('cohort_id').user_id.nunique()

cohort_id
1.0      18
2.0       3
4.0       1
5.0       1
6.0       2
7.0       6
8.0       5
9.0       1
11.0      5
12.0      5
13.0     10
14.0     13
15.0      5
16.0      8
17.0     14
18.0     14
19.0     17
21.0     28
22.0     35
23.0     33
24.0     34
25.0     26
26.0     20
27.0     29
28.0     51
29.0     25
31.0     28
32.0     22
33.0     29
34.0     23
51.0     27
52.0     28
53.0     29
55.0     21
56.0     24
57.0     29
58.0     23
59.0     29
61.0     23
62.0     33
132.0    23
133.0    17
134.0    20
135.0    31
137.0    21
138.0    37
139.0    26
Name: user_id, dtype: int64

In [78]:
active_stu.groupby('cohort_id').user_id.nunique()

cohort_id
22.0     2
23.0     1
24.0     2
25.0     1
26.0     1
27.0     1
29.0     2
31.0     3
32.0     2
33.0     2
34.0     2
55.0     1
56.0     3
58.0     1
59.0     2
61.0     1
62.0     4
132.0    1
133.0    3
134.0    1
135.0    1
137.0    1
Name: user_id, dtype: int64

In [91]:
df.user_id[(df.cohort_id==22.0) & (df.is_active==False)].nunique()

23

In [80]:
active_stu.user_id[active_stu.cohort_id==22.0]

date
2018-03-14     5
2018-01-26    13
Name: user_id, dtype: int64

In [82]:
active_stu[active_stu.user_id ==5]

,time,endpoint,user_id,cohort_id,ip,cohort,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,is_active
date,,,,,,,,,,,,,,
2018-03-14,01:20:12,java-i,5,22.0,24.155.251.203,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,True


### 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

In [143]:
df.ip.value_counts().idxmax()

'97.105.19.58'

In [97]:
df['pages'] = df['endpoint'].resample('d').count()

In [98]:
df.head()

,time,endpoint,user_id,cohort_id,ip,cohort,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,is_active,pages
date,,,,,,,,,,,,,,,
2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,False,531
2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,False,531
2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,False,531
2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,False,531
2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,True,531


In [114]:
def prep(df, user):
    '''
    This function takes in a df and a single user. 
    It converts the date column to a datetime object and sets that as the index.
    It also resamples daily counts for number of page views.
    '''
    df = df[df.user_id == user]
    pages = df['endpoint'].resample('d').count()
    return pages

def compute_pct_b(pages, span, weight, user):
    '''
    This function takes in the pages, the number of days, the weight to calculate the standard deviation and the user.
    It calculates the mid, upper and lower bollinger bands and concatenates the upper and lower bands.
    It then creates a new dataframe with the number of page view, midband, upper and lower bands.
    It calculate the %b and adds a user id column to the df and then returns the df.
    '''
    midband = pages.ewm(span=span).mean()
    stdev = pages.ewm(span=span).std()
    ub = midband + stdev * weight
    lb = midband - stdev * weight
    bb = pd.concat([ub, lb], axis=1)
    my_df = pd.concat([pages, midband, bb], axis=1)
    my_df.columns = ['pages', 'midband', 'ub', 'lb']
    my_df['pct_b'] = (my_df['pages'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
    my_df['user_id'] = user
    return my_df

def find_anomalies(df, user, span, weight):
    '''
    This function takes in a df, user id, number of days as span and the weight to compute %b. 
    It then prepares the df by calling the prep function.
    It also creates a new dataframe by calling the compute pct b function.
    It returns a subset of the df where %b is greater than 1.
    '''
    pages = prep(df, user)
    my_df = compute_pct_b(pages, span, weight, user)
    return my_df[my_df.pct_b > 1]

In [115]:
# test on a single user

user = 68
span = 30
weight = 3

# initialize empty df
anomalies = pd.DataFrame()

# call function to create a new df with anomalies for the user 
user_df = find_anomalies(df, user, span, weight)

# concatenate the above df created with the empty anomalies df
anomalies = pd.concat([anomalies, user_df], axis=0)

In [116]:
anomalies

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2018-05-04,67,7.035731,60.259707,-46.188246,1.063320,68
2018-06-25,50,4.811412,45.265482,-35.642658,1.058517,68
2018-08-20,17,1.240542,15.590082,-13.108997,1.049128,68
2018-10-16,13,0.898678,10.970672,-9.173316,1.100741,68
2018-11-15,27,1.863468,22.364493,-18.637557,1.113055,68
2019-01-30,21,1.407805,17.354013,-14.538403,1.114321,68
2019-03-25,24,1.622558,19.932129,-16.687012,1.111086,68
2019-06-27,9,0.587115,7.439000,-6.264770,1.113910,68
2019-07-10,14,1.554444,13.962988,-10.854099,1.001491,68


In [117]:
# loop through all users to find anomalies for each
span = 30
weight = 3.5

anomalies = pd.DataFrame()
for u in list(df.user_id.unique()):
    user_df = find_anomalies(df, u, span, weight)
    anomalies = pd.concat([anomalies, user_df], axis=0)

In [118]:
anomalies

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2019-01-29,44,3.623334,42.081344,-34.834677,1.024945,1
2019-07-08,101,8.224728,96.466493,-80.017037,1.025688,1
2018-10-15,9,0.581421,8.457413,-7.294571,1.034446,3
2019-01-09,4,0.262470,3.802115,-3.277175,1.027953,3
2019-04-06,2,0.129825,1.889712,-1.630061,1.031334,3
2018-07-19,38,3.202804,37.692086,-31.286478,1.004464,5
2018-06-02,74,6.685267,72.357092,-58.986557,1.012508,6
2019-03-18,17,1.096774,15.964034,-13.770485,1.034841,6
2019-06-27,5,0.322605,4.695345,-4.050135,1.034836,7


In [119]:
# look at value counts for pages in anomalies without sorting the counts
anomalies.pages.value_counts(sort = False)

1      43
2      34
3      65
4      74
5      54
6      46
7      45
8      47
9      40
10     30
11     33
12     21
13     29
14     21
15     22
16     21
17     17
18     20
19     17
20     15
21     14
22     11
23     12
24     11
25      9
26      9
27     11
28     12
29      7
30     10
31      1
32     11
33      9
34     12
35      5
36      9
37      3
38      4
39      1
40      7
41      5
42      3
43      3
44      5
45      3
46      3
47      2
48      3
49      1
50      2
52      4
53      3
54      2
55      2
56      2
57      2
58      4
59      2
60      1
61      1
62      3
64      1
66      2
67      2
69      1
72      1
73      3
74      2
75      1
76      1
78      4
81      1
82      1
84      1
93      1
94      2
95      3
101     2
104     2
107     1
108     1
113     1
115     1
117     1
122     1
128     1
129     1
153     1
165     1
179     1
192     1
197     1
272     1
Name: pages, dtype: int64

The highest number of page views is 272 so let's investigate that.

In [126]:
anomalies[anomalies.pages == 197]

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2018-06-02,197,16.419703,187.836552,-154.997147,1.026729,138


In [127]:
anomalies[anomalies.user_id == 138]

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2018-06-02,197,16.419703,187.836552,-154.997147,1.026729,138
2018-12-15,3,0.193901,2.837079,-2.449278,1.030819,138
2019-06-09,4,0.258066,3.756252,-3.240120,1.034839,138
2019-09-16,5,0.323617,4.699978,-4.052743,1.034278,138


In [132]:
df[(df.user_id == 138) & (df.index == '2018-06-02')]

,time,endpoint,user_id,cohort_id,ip,cohort,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,is_active,pages
date,,,,,,,,,,,,,,,
2018-06-02,15:03:29,/,138,22.0,108.65.244.91,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,False,447
2018-06-02,15:04:28,content/html-css,138,22.0,108.65.244.91,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,False,447
2018-06-02,15:04:31,content/html-css,138,22.0,108.65.244.91,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,False,447
2018-06-02,15:04:33,content/html-css/introduction.html,138,22.0,108.65.244.91,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,False,447
2018-06-02,15:04:54,content/html-css/elements.html,138,22.0,108.65.244.91,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,False,447
2018-06-02,15:04:56,content/html-css/forms.html,138,22.0,108.65.244.91,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,False,447
2018-06-02,15:05:00,content/html-css/css-i,138,22.0,108.65.244.91,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,False,447
2018-06-02,15:05:01,content/html-css/css-i/introduction.html,138,22.0,108.65.244.91,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,False,447
2018-06-02,15:05:03,content/html-css/css-i/selectors-and-propertie...,138,22.0,108.65.244.91,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,False,447


In [133]:
df[(df.user_id == 138) & (df.index == '2018-12-15')]

,time,endpoint,user_id,cohort_id,ip,cohort,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,is_active,pages
date,,,,,,,,,,,,,,,
2018-12-15,10:34:45,/,138,22.0,108.65.244.91,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,False,70
2018-12-15,10:34:51,html-css,138,22.0,108.65.244.91,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,False,70
2018-12-15,10:34:54,html-css/elements,138,22.0,108.65.244.91,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,False,70


In [120]:
# look at a subset of anomalies where the number of page views is equal to 272
anomalies[anomalies.pages == 272]

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2019-03-03,272,24.721632,266.780128,-217.336864,1.010782,341


In [121]:
# pull a subset of data where user id is 341
anomalies[anomalies.user_id == 341]

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2019-03-03,272,24.721632,266.780128,-217.336864,1.010782,341
2020-02-18,3,0.193548,2.817185,-2.430088,1.034840,341
2020-04-21,104,6.928145,97.719968,-83.863678,1.034585,341
2021-02-09,10,0.659630,9.508602,-8.189342,1.027766,341


In [123]:
sus_user = df.loc['2019-03-02':'2019-04-22']

In [125]:
sus_user[sus_user.user_id == 341]

,time,endpoint,user_id,cohort_id,ip,cohort,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,is_active,pages
date,,,,,,,,,,,,,,,
2019-03-03,21:14:08,/,341,29.0,173.174.243.231,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,549
2019-03-03,21:14:14,jquery,341,29.0,173.174.243.231,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,549
2019-03-03,21:14:27,jquery/essential-methods/traversing,341,29.0,173.174.243.231,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,549
2019-03-03,21:15:25,jquery/effects,341,29.0,173.174.243.231,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,549
2019-03-03,22:52:05,html-css,341,29.0,204.44.112.76,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,549
2019-03-03,22:52:06,javascript-i,341,29.0,204.44.112.76,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,549
2019-03-03,22:52:06,java-i,341,29.0,204.44.112.76,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,549
2019-03-03,22:52:06,java-ii,341,29.0,204.44.112.76,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,549
2019-03-03,22:52:06,javascript-ii,341,29.0,204.44.112.76,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,549


In [122]:
df[df.user_id == 341]

,time,endpoint,user_id,cohort_id,ip,cohort,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,is_active,pages
date,,,,,,,,,,,,,,,
2019-01-22,15:23:24,/,341,29.0,97.105.19.58,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,769
2019-01-22,15:25:51,toc,341,29.0,97.105.19.58,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,769
2019-01-22,15:26:44,html-css,341,29.0,97.105.19.58,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,769
2019-01-22,15:28:33,html-css/introduction,341,29.0,97.105.19.58,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,769
2019-01-22,15:29:29,html-css/elements,341,29.0,97.105.19.58,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,769
2019-01-22,15:31:24,html-css/introduction,341,29.0,97.105.19.58,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,769
2019-01-22,15:31:32,html-css/forms,341,29.0,97.105.19.58,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,769
2019-01-22,15:33:40,html-css/css-i,341,29.0,97.105.19.58,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,769
2019-01-22,15:33:43,html-css/css-i/positioning,341,29.0,97.105.19.58,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2,True,769


### 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?

In [137]:
anomalies[anomalies.pages == 1]

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2019-11-12,1,0.064527,0.939245,-0.810190,1.034729,15
2019-05-21,1,0.064519,0.939144,-0.810106,1.034790,19
2020-03-08,1,0.064516,0.939062,-0.810030,1.034840,31
2018-07-04,1,0.068298,0.982751,-0.846154,1.009431,41
2019-04-10,1,0.064516,0.939061,-0.810029,1.034840,65
2019-04-03,1,0.064516,0.939061,-0.810028,1.034841,69
2019-07-11,1,0.064516,0.939067,-0.810035,1.034836,96
2020-04-19,1,0.064516,0.939070,-0.810037,1.034835,116
2020-10-05,1,0.064516,0.939061,-0.810029,1.034840,123


In [150]:
df.cohort_id.max()

139.0

In [151]:
df.head()

,time,endpoint,user_id,cohort_id,ip,cohort,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,is_active,pages
date,,,,,,,,,,,,,,,
2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,False,531
2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,False,531
2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,False,531
2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,False,531
2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,True,531


In [ ]:
df.groupby('cohort_')